# Flask Docker Compose

### Introduction

So far we have been using docker to manage and deploy single services.  For example, managing a jupyter notebook could be considered a single service.  Or scraping a website could be considered a single service.  

However when we move onto something like a web application, there are multiple services we can think of operating.  For example, just hosting the backend of the website could be a service, and the databasem could be another service, as could the task of gathering data, by say scraping other websites could be a service.   

With docker, we already know how to build these separate services.  The benefit of docker compose is that it is an easy mechanism to allow these services to talk to each other.

### Using Docker Compose

To get started let's just use docker-compose with a single service, our flask application.  We can do so by creating a `docker-compose.yml` file in `services/` folder.

`services/docker-compose.yml`

```python
version: '3.7'
services:
  users:
    build:
      context: ./web-app
      dockerfile: Dockerfile
    ports:
      - 3000:5000
    environment:
      - FLASK_APP=project/__init__.py
      - FLASK_ENV=development
      - APP_SETTINGS=project.config.DevelopmentConfig
```

With this work, we can accomplish the same work that we accomplished with running our docker run command of:

`docker run -p 3000:5000 -e FLASK_APP=project/__init__.py jek2141/flask-api-docker`

Ok, let's run it.

`docker-compose up`

```
Starting services_users_1 ... done
Attaching to services_users_1
users_1  |  * Serving Flask app "project/__init__.py" (lazy loading)
```

If we visit, `0.0.0.0:3000` we'll see that this accomplished the same as our longer line.

### Understanding our docker-compose file

Now looking at the `docker-compose.yml` file, a lot of it is fairly intuitive.

```python
version: '3.7'
services:
  users:
    build:
      context: ./web-app
      dockerfile: Dockerfile
    ports:
      - 3000:5000
    environment:
      - FLASK_APP=project/__init__.py
      - FLASK_ENV=development
      - APP_SETTINGS=project.config.DevelopmentConfig
```

Let's break it down:

The `version` specifies the version number of docker-compose that is being used.  Then we specify each of the `services`.  For us, we have a `users` service, where we describe the `build`, `ports` and the `environment`.  The `build` key just specifies the file used to build the Dockerfile.  Here, relative to the location of the `docker-compose.yml` file, it's `./users/Dockerfile`.  The ports, are where we specify the mapping, 3000:5000, and `environment` is used for the environment variables.

The hyphens that we see under `environment` is to indicate that what follows is a list, and we precede each element in the list with a `-`.  So we have a list of environment variables, so far of `FLASK_APP` and `FLASK_ENV`.  Using the `-` to indicate list elements is a yaml convention.

So using this file, we have properly translated arguments of our `docker run` command to components of our `docker-compose.yml` file. 

Now, let's make use of the `APP_SETTINGS` environment variable.  We use this to load up different setting configurations based on if we are in development or production.  To put this into effect, we update our `__init__.py` file to the following:

```python
import os
from flask import Flask, jsonify
from flask_restful import Resource, Api

app = Flask(__name__)

api = Api(app)

app_settings = os.getenv('APP_SETTINGS')  
app.config.from_object(app_settings)      
```

And this will load up the relevant settings defined in the `project/config.py` file.

### Bind Mounting with Docker Compose

```python
version: '3.7'
services:
  web:
    build:
      context: ./web-app
      dockerfile: Dockerfile
    ports:
      - 3000:5000
    environment:
      - FLASK_APP=project/__init__.py
      - FLASK_ENV=development
      - APP_SETTINGS=project.config.DevelopmentConfig
    volumes:
      - .:/web/flask
```

### Summary